## Query가 아닌 Document 형태로 input이 될 경우

In [1]:
from pymilvus import connections, utility, FieldSchema, CollectionSchema, DataType, Collection

QUERY_MODEL = 'solar-embedding-1-large-query'
PASSAGE_MODEL = 'solar-embedding-1-large-passage'

DENSE_FIELD = "dense_vector"
DENSE_DIMENSION = 4096 
DENSE_INDEX = {"index_type" : "FLAT", "metric_type" : "IP"}

SPARSE_FILED = "sparse_vector"
SPARSE_INDEX = {"index_type" : "SPARSE_INVERTED_INDEX", "metric_type" : "IP"}

# Collection 접속 및 생성
def milvus_connect(collection_name):
    # Milvus 설정
    MILVUS_HOST = '127.0.0.1'
    MILVUS_PORT = '19530'     

    # Milvus 연결
    connections.connect(host=MILVUS_HOST, port=MILVUS_PORT)
    if connections:
        print("Milvus connected")
    else:
        exit()

    # 컬렉션 존재 여부 확인 및 생성
    if utility.has_collection(collection_name):
        collection = Collection(collection_name)
        print(f"Collection '{collection_name}' loaded.")
    else:
        print(f"Collection '{collection_name}' does not exist.")
        print(f"Create '{collection_name}' Collection.")
        
        field_args = [
            FieldSchema(name='id', dtype=DataType.VARCHAR, is_primary=True, max_length=100),        # split된 block의 고유 ID
            FieldSchema(name='title', dtype=DataType.VARCHAR, max_length=2000),
            FieldSchema(name='date', dtype=DataType.INT64),
            FieldSchema(name='content', dtype=DataType.VARCHAR, max_length=5000),
            FieldSchema(name='NER', dtype=DataType.VARCHAR, max_length=3000),
            FieldSchema(name=DENSE_FIELD, dtype=DataType.FLOAT_VECTOR, dim=DENSE_DIMENSION),
            FieldSchema(name=SPARSE_FILED, dtype=DataType.SPARSE_FLOAT_VECTOR)
        ]
        
        # 스키마 정의
        schema = CollectionSchema(fields=field_args)
        
        # 컬렉션 생성
        collection = Collection(name=collection_name, schema=schema)
        
        # 인덱스 생성
        collection.create_index("dense_vector", DENSE_INDEX)
        collection.create_index("sparse_vector", SPARSE_INDEX)

        collection.flush()

        print(f"Collection '{collection_name}' created and loaded.")
    # 생성된 컬렉션 반환
    return collection

# Collection 제거용
def drop_collection(collection_name):
    MILVUS_HOST = '127.0.0.1'
    MILVUS_PORT = '19530' 
    connections.connect(host=MILVUS_HOST, port=MILVUS_PORT)
    if utility.has_collection(collection_name):
        utility.drop_collection(collection_name)
        print(f"Drop {collection_name} Complete.")
    else:
        print(f"Can't find {collection_name}.")

In [2]:
donga = milvus_connect("Donga")
print(donga)

donga_800 = milvus_connect("Donga_800")
print(donga_800)

Milvus connected
Collection 'Donga' loaded.
<Collection>:
-------------
<name>: Donga
<description>: 
<schema>: {'auto_id': False, 'description': '', 'fields': [{'name': 'id', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 100}, 'is_primary': True, 'auto_id': False}, {'name': 'title', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 2000}}, {'name': 'date', 'description': '', 'type': <DataType.INT64: 5>}, {'name': 'content', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 5000}}, {'name': 'NER', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 3000}}, {'name': 'dense_vector', 'description': '', 'type': <DataType.FLOAT_VECTOR: 101>, 'params': {'dim': 4096}}, {'name': 'sparse_vector', 'description': '', 'type': <DataType.SPARSE_FLOAT_VECTOR: 104>}], 'enable_dynamic_field': False}

Milvus connected
Collection 'Donga_800' loaded.
<Collection>:
-------------
<name>: Donga_800
<desc

## Donga 세팅

In [3]:
# Sparse Embedding : Custom BM25
from model.bm25_model import Custom_BM25, Kiwi_Tokenizer

tokenizer = Kiwi_Tokenizer("model/custom_dict.txt")
custom_bm25 = Custom_BM25(corpus=[], tokenizer=tokenizer)
custom_bm25.load("model/bm25_model.json")
print(custom_bm25)

# Dense Embedding : Upstage/solar-embedding-1-large
from langchain_upstage import UpstageEmbeddings
from dotenv import load_dotenv
import os

load_dotenv()

dense_embedding_func = UpstageEmbeddings(
    model="solar-embedding-1-large-passage",
    upstage_api_key=os.getenv("UPSTAGE_API_KEY")
)

c:\Users\kidong\anaconda3\envs\milvus_hybrid\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [4]:
from langchain_milvus.retrievers import MilvusCollectionHybridSearchRetriever
from pymilvus import WeightedRanker

def hybrid_search(query : str, filter : str = None):
    sparse_search_params = {"metric_type": "IP"}
    dense_search_params = {"metric_type": "IP", "params": {}}

    params = {
        "collection" : donga,
        "rerank" : WeightedRanker(0.7, 0.3),
        "anns_fields" : [DENSE_FIELD, SPARSE_FILED],
        "field_embeddings" : [dense_embedding_func, custom_bm25],
        "field_search_params" : [dense_search_params, sparse_search_params],
        "top_k" : 20,
        "text_field" : 'content',
    }
    if filter:
        params['field_exprs'] = [filter, filter] # 검색 조건 추가(dense와 sparse에 동일하게 적용)
    retriever = MilvusCollectionHybridSearchRetriever(**params)
    
    retrieved_docs = retriever.invoke(query)
    data = []
    unique_id = []
    for retreived_doc in retrieved_docs:
        doc_id = retreived_doc.metadata['id'].split("_")[0] # 기사 고유 ID 추출
        if doc_id not in unique_id: # 해당 기사가 아직 반환 리스트에 없는 경우만
            data.append(retreived_doc)
            unique_id.append(doc_id)
        if len(data) == 5: break # 상위 5개만 반환
    return data

## Donga_800 세팅

In [5]:
from openai import OpenAI
import numpy as np

INDEX_PARAMS = {
    'metric_type': 'COSINE',     # "L2", "IP", "COSINE" 
    'index_type': "IVF_FLAT",    # "IVF_FLAT","IVF_SQ8", "IVF_PQ", "HNSW", "ANNOY"
    'params': {"nlist": 128},
}

client = OpenAI(
    api_key = os.getenv("UPSTAGE_API_KEY"),
    base_url = "https://api.upstage.ai/v1/solar"
)

def vector_search(collection, query, index_params, option=None):
    # Query embedding 생성
    query_embedding = client.embeddings.create(
        model="solar-embedding-1-large-passage",
        input=query
    ).data[0].embedding
    query_embedding = np.asarray(query_embedding)
    
    # 공통 search 파라미터 설정
    search_params = {
        "data": [query_embedding],
        "anns_field": 'embedding',
        "param": index_params,
        "limit": 5,
        "output_fields": ['title', 'date', 'content', 'link']
    }

    # 옵션이 있는 경우 expr 추가
    if option:
        search_params["expr"] = option

    # 검색 실행
    result = collection.search(**search_params)

    # 결과 처리
    result_list = []
    for hits in result:
        for hit in hits:
            result_list.append(hit.to_dict())
    
    return result_list

# Test

In [12]:
# 출처 기사 : 1평에 24명 ‘빽빽’… “미등록 공연장, 제2 이태원 참사 우려”, 2024.08.02 (https://n.news.naver.com/mnews/article/020/0003579874)
# 원문 기사는 1800자 정도, 이를 chatgpt 4o를 이용하여 700 ~ 800자 분량으로 요약
passage1 = """
지난달 29일 서울 성동구 복합문화공간 에스팩토리에서 압사 우려로 공연이 중단됐으며, 당시 1㎡당 최대 7명이 밀집해 있었습니다. 이는 2022년 10월 이태원 참사 당시 밀집도에 근접하는 수준이었습니다. 에스팩토리는 공연법상 등록되지 않은 공연장으로, 수용 인원 규제가 없어 관리 사각지대에 놓여 있었습니다. 전문가들은 공연장 3층의 밀집도를 분석한 결과, 1인당 점용 면적이 손바닥 크기인 0.14∼0.25㎡에 불과했다고 밝혔습니다. 현행법상 바닥 면적이 0.5㎡ 이하면 위험한 상태로 분류됩니다. 에스팩토리의 공연 주관사 측은 자체 기준으로 관객 수를 정했다고 밝혔지만, 전문가들은 이 같은 미등록 공연장에서 대형 사고가 발생할 수 있다고 경고합니다. 향후 인천 송도 록페스티벌, 워터밤 등 대규모 공연들이 예정되어 있어 대책 마련이 시급합니다. 전문가들은 공연장 내 수용 인원을 안전과 직결된 문제로 보고, 법적 규정을 만들어야 한다고 주장합니다. 공연 도중 인파가 무대 쪽으로 몰리는 현상과 안전요원의 지시가 통하지 않는 상황이 반복될 수 있어 철저한 인파 관리 대책이 필요합니다. 지난 6월 강원에서 열린 페스티벌에서도 무대 근처 스탠딩 구역에서 관객들이 압박을 받아 어려움을 겪었으며, 안전요원의 지시가 잘 통제되지 않았습니다. 이러한 문제를 해결하기 위해 공연장의 모든 구역 내 상시 안전 공간을 확보하고 세심한 관리 대책을 마련해야 한다는 지적이 나옵니다.
"""

In [15]:
hybrid_search(passage1, filter='date >= 20240601') # donga_800과 동일한 데이터 기반으로 하기 위해 6월~7월 기사만으로 구성

[Document(metadata={'id': '126048332_0', 'title': "강풍에 무대 조명 '와르르'…아찔했던 싸이 흠뻑쇼", 'date': 20240722, 'NER': '경기 과천 가수 싸이 위 아래 싸이 싸이 흠뻑쇼  SUMMER SWAG 썸머 스웨그  아래 피네이션'}, page_content='경기 과천 서울대공원에서 열렸던 가수 싸이의 \'흠뻑쇼\'가 폭우와 낙뢰 등으로 중단된 가운데, 당시 무대 위 조명 장치가 5m 아래로 떨어지는 등 아찔한 상황이 발생한 것으로 드러났다. 싸이는 지난 20일 서울대공원에서 \'싸이 흠뻑쇼 SUMMER SWAG 2024(썸머 스웨그 2024)\'를 진행했다. 공연 시작 전부터 비가 내렸다. 공연 시작 후에는 비바람이 몰아치고 낙뢰까지 떨어졌다. 강풍으로 인해 흔들리던 조명이 무대 아래로 떨어지기도 했다. 스태프와 관객 중 다친 사람은 없는 것으로 알려졌다. 결국 안전상의 이유로 공연은 1시간여 만에 중단됐다. 소속사 피네이션은 "강한 돌풍을 동반한 국지성 폭우 및 낙뢰 등 악천후로 인해 정상적인 공연 진행이 어렵다고 판단돼 관객과 스태프의 안전을 최우선으로 하고자 공연 중단 결정을 내리게 됐다"고 밝혔다. 이어 "이날 공연 티켓은 예매처를 통해 순차적으로'),
 Document(metadata={'id': '125280077_0', 'title': "[글로벌 현장을 가다/김철중]K가수-드라마 환호 늘었지만… 중 '한한령' 전면해제까진 '먼 길'", 'date': 20240604, 'NER': '중국인 중국 베이징시 차오양구 엘리베이터 중국인 가족 한국어 대만 샤위톈 한국 오뮤 OUBA 중국인 한국문화원 신종 코로나바이러스 감염증 코로나19 중국 한국 음악가수 톈진시'}, page_content="'K컬처 해방구' 발길 잦아진 중국인들 <1일 오후 7시경 중국 베이징시 차오양구에 있는 주중 한국문화원. 엘리베이터를 타고 지하 1층 공연장으로 내려가니 이미 자리가 사람들로 꽉 들어차 있었다. 200석

In [14]:
vector_search(donga_800, passage1, INDEX_PARAMS)

[{'id': 451472232637364719,
  'distance': 0.48154762387275696,
  'entity': {'title': '소음 차단-대화 금지…도심 속 힐링공간 인기',
   'date': 20240606,
   'content': '서울 마포구 마포아트센터 소극장 플레이맥. 어둑한 극장 안으로 들어가자 베토벤의 음악이 나지막히 깔리고 있었다. 좌석에 앉으니 차분해진다. 도시의 소음도 들리지 않는다. 주변 사람들도 눈을 감고 음악 자체에 집중하거나 책을 읽으면서 힐링 타임을 즐기는 듯했다. 바쁘고 정신없이 돌아가는 도시의 일상 속에서 속도를 늦추는 공간을 사람들이 부러 찾고 있다. 소극장은 음악감상실로 바꿨고 대화 금지를 앞세운 침묵 카페 등도 인기다. 마포아트센터는 지난달 29일 도심 속 음악감상실인 음악 공간 플레이 리스트를 선보였다. 오후 1시부터 5시까지 개방된 소극장에서 가 선곡한 음악을 누구나 감상할 수 있도록 한 것. 정해진 시간 내에 자유롭게 입퇴장이 가능하고 좌석도 자율 착석제지만 단 하나 조건이 있다. 음악을 들으며 홀로 사색할 수 있도록 다른 관객과의 자리를 띄워 앉아야 하는 것. 러브 아다지오 죽음 같은 사랑으로 진행한 클래식 음악 감상 1회차에선 신청자 40여 명이 자유롭게 음악을 감상하고 돌아갔다. 6월에는 흔들리는 사람들 스윙과 블루노트 재즈란 주제로 26일 감상회를 열 예정이다. 꽉 막힌 박스형 공연장이 아닌 한강공원 잔디밭에서 불어오는 바람을 느끼며 클래식 악기의 선율을 듣는 공연도 눈길을 끈다. 서울시립교향악단은 7 8일 서울 여의도 한강공원 멀티플라자 이벤트광장에서 강변음악회를 연다. 2000석 규모의 야외 객석엔 누구나 선착순으로 앉을 수 있고 좌석이 없더라도 무대 주변 잔디밭에서 음악을 감상할 수 있다. 영화 인디아나 존스 . 미션 삽입곡 등 대중에게 친숙한 음악을 선보인다. 세대 사이에선 대화 금지 ',
   'link': 'https://www.donga.com/news/Culture/a

In [16]:
hybrid_search(passage1) # 필터없이 2024년 전체 기사를 기반으로

[Document(metadata={'id': '126048332_0', 'title': "강풍에 무대 조명 '와르르'…아찔했던 싸이 흠뻑쇼", 'date': 20240722, 'NER': '경기 과천 가수 싸이 위 아래 싸이 싸이 흠뻑쇼  SUMMER SWAG 썸머 스웨그  아래 피네이션'}, page_content='경기 과천 서울대공원에서 열렸던 가수 싸이의 \'흠뻑쇼\'가 폭우와 낙뢰 등으로 중단된 가운데, 당시 무대 위 조명 장치가 5m 아래로 떨어지는 등 아찔한 상황이 발생한 것으로 드러났다. 싸이는 지난 20일 서울대공원에서 \'싸이 흠뻑쇼 SUMMER SWAG 2024(썸머 스웨그 2024)\'를 진행했다. 공연 시작 전부터 비가 내렸다. 공연 시작 후에는 비바람이 몰아치고 낙뢰까지 떨어졌다. 강풍으로 인해 흔들리던 조명이 무대 아래로 떨어지기도 했다. 스태프와 관객 중 다친 사람은 없는 것으로 알려졌다. 결국 안전상의 이유로 공연은 1시간여 만에 중단됐다. 소속사 피네이션은 "강한 돌풍을 동반한 국지성 폭우 및 낙뢰 등 악천후로 인해 정상적인 공연 진행이 어렵다고 판단돼 관객과 스태프의 안전을 최우선으로 하고자 공연 중단 결정을 내리게 됐다"고 밝혔다. 이어 "이날 공연 티켓은 예매처를 통해 순차적으로'),
 Document(metadata={'id': '124129189_0', 'title': '"죽은 척 엎드려있어" "피투성이" 생존자들이 전한 증언', 'date': 20240324, 'NER': '모스크바 러시아 모스크바 한 러시아 총 뒤 총소리 아리나 영국 가디언 소총 아리나 옆'}, page_content='최소 133명 사망한 모스크바 공연장 테러 러시아 모스크바 외곽의 한 대형 공연장에서 22일(이하 현지시간) 테러 참사가 발생한 가운데 현장에서 목숨을 건진 생존자들의 증언이 이어졌다. 당일 7000명에 달하는 인파가 러시아 록밴드 피크닉의 공연을 보기 위해 공연장을 찾았고, 테러범들은 공연이 시작되기 전 무차별적으로 총을 난사한 

In [17]:
# 출처 기사 : 끝이 보이는 고금리 시대… 美연준, 9월 인하 시사, 2024.08.02 (https://n.news.naver.com/mnews/article/020/0003579902)
# 원문 기사는 4000자 정도, 이를 chatgpt 4o를 이용하여 700 ~ 800자 분량으로 요약
passage2 = """
미국 연방준비제도(Fed)가 9월 금리 인하를 강력히 시사하면서 글로벌 고금리 사이클이 2년 반 만에 마무리될 전망이다. 지난달 31일 연준은 연방공개시장위원회(FOMC)에서 기준금리를 동결했으나, 제롬 파월 의장은 9월 회의에서 금리 인하가 논의될 수 있다고 밝혔다. 시장은 9월 금리 인하 가능성을 거의 100%로 보고 있으며, 물가 안정과 고용시장 둔화가 그 배경이다. 6월 개인소비지출(PCE) 상승률은 전년 대비 2.5%로 안정됐고, 실업률은 4.1%로 상승했다. 미국 외 주요국들도 금리 인하에 나서고 있다. 캐나다는 두 달 연속 금리를 인하했고, 유럽중앙은행(ECB)과 중국, 영국도 금리를 낮췄다. 이러한 금리 인하는 글로벌 경제에 큰 변화를 가져올 전망이다. 금융시장에서는 자산 가치가 상승 압력을 받을 수 있고, 민간 소비와 기업 투자도 늘어날 가능성이 있다. 강달러 현상이 완화되면서 아시아 등 다른 나라의 통화 가치가 반등할 여지도 있다. 한국은행도 금리 인하 가능성을 검토 중이다. 내수 경기 침체와 물가 상승세 둔화로 한은의 10월 금리 인하설이 힘을 얻고 있다. 이창용 한은 총재는 금리 인하 논의를 시작했다고 밝혔다. 한은은 2021년 8월부터 금리를 인상해 현재 3.50%를 유지해왔으며, 고금리로 인해 가계 이자상환 부담이 늘고 내수와 부동산 경기가 둔화되는 등 부작용이 있었다. 정부와 여당도 한은에 금리 인하를 요구하고 있다. 그러나 금리 인하가 부동산 시장을 자극할 수 있다는 점은 한은에 부담이다. 부동산값 상승이 은행권 주택담보대출을 늘려 가계부채가 증가할 수 있기 때문이다. 미국 대선과 중동 전쟁, 달러화 강세, 국제 유가 급등 등으로 국내 물가가 다시 불안해질 가능성도 있다. 전문가들은 한은이 10월 금리 인하에 나설 가능성을 예상하면서도, 부동산 대책 효과 등을 살핀 뒤 결정할 것으로 본다.
"""

In [18]:
hybrid_search(passage2, filter='date >= 20240601') # donga_800과 동일한 데이터 기반으로 하기 위해 6월~7월 기사만으로 구성

[Document(metadata={'id': '125425689_0', 'title': '미연준, 기준금리 올해 1차례 인하 시사', 'date': 20240614, 'NER': '미국 연방준비제도 Fed 유럽 캐나다 미국 연방공개시장위원회 미 제롬 파월 의장'}, page_content='금리 7연속 동결… 5.5% 유지 파월 \'9월 인하\'도 배제 안해 미국 중앙은행 연방준비제도(Fed·연준)가 12일(현지 시간) 기준금리를 동결하고 연말까지 한 차례의 금리 인하를 시사했다. 유럽과 캐나다가 금리 인하로 \'피벗\'(정책 전환)에 나섰지만, 미국은 9월 금리 인하 가능성을 낮추며 고금리 유지에 무게를 뒀다. 연준은 이날 연방공개시장위원회(FOMC) 정례회의를 열고 "7차례 연속 금리를 동결해 미 기준금리를 5.25~5.50%로 유지한다"고 밝혔다. 연말 금리 중간값은 5.1%(5.0~5.25%)로 현 금리보다 0.25%포인트 낮은 수준이다. 이날 오전 발표된 5월 소비자물가지수(CPI) 상승률이 3.3%로 시장 전망치(3.4%)를 하회하는 등 물가상승률 둔화 시그널이 나왔지만, 연준은 기존 3차례 인하에서 1차례 인하로 인하 전망 폭을 오히려 축소한 것이다. 제롬 파월 연준 의장은 '),
 Document(metadata={'id': '125343356_0', 'title': 'ECB-캐나다 \'피벗\'에도… "한-미 금리인하 예상보다 늦춰질듯"', 'date': 20240609, 'NER': '미 유럽중앙은행 ECB 캐나다 한국 미국 ECB 캐나다 유럽 캐나다 한국 미국 블룸버그 씨티그룹 JP모건 미국 연방준비제도 Fed'}, page_content='양국 경제지표 양호-고물가 이어져 월가 "미 9월이나 12월 인하 예상" 최근 유럽중앙은행(ECB)과 캐나다 중앙은행이 기준금리를 인하했지만, 한국과 미국의 피벗(통화정책 전환)은 시장의 예상보다 늦춰질 것이라는 전망이 나오고 있다. 9일 외신 등에 따르면 ECB는 6일(현지 시간) 기준금리를 0.25%

In [19]:
vector_search(donga_800, passage2, INDEX_PARAMS)

[{'id': 451472232637367795,
  'distance': 0.7740257382392883,
  'entity': {'title': '美연준, 기준금리 올해 1차례 인하 시사',
   'date': 20240614,
   'content': '금리 7연속 동결 5.5% 유지 파월 9월 인하도 배제 안해 미국 중앙은행 연방준비제도연준가 12일현지 시간 기준금리를 동결하고 연말까지 한 차례의 금리 인하를 시사했다. 유럽과 캐나다가 금리 인하로 피벗정책 전환에 나섰지만 미국은 9월 금리 인하 가능성을 낮추며 고금리 유지에 무게를 뒀다. 연준은 이날 연방공개시장위원회 정례회의를 열고 7차례 연속 금리를 동결해 미 기준금리를 5.255.50%로 유지한다고 밝혔다. 연말 금리 중간값은 5.1%5.05.25%로 현 금리보다 0.25%포인트 낮은 수준이다. 이날 오전 발표된 5월 소비자물가지수 상승률이 3.3%로 시장 전망치3.4%를 하회하는 등 물가상승률 둔화 시그널이 나왔지만 연준은 기존 3차례 인하에서 1차례 인하로 인하 전망 폭을 오히려 축소한 것이다. 제롬 파월 연준 의장은 이날 기자회견에서 지표는 인플레이션 둔화에 진전을 보여줬지만 한 번 좋은 지표가 나왔다고 바로 움직일 순 없다고 말했다. 다만 두 차례 인하도 가능하다고 덧붙여 9월 인하 가능성을 완전히 배제하진 않았다. 시카고상품거래소 페드워치에 따르면 미 기준금리 선물 투자자들은 이날 오전 발표 직후 9월 인하 가능성을 약 70%까지 내다봤으나 파월 기자회견 이후 60%로 낮췄다. 미 연준이 기준 금리를 연속 동결함으로써 한국과의 금리 격차는 최대 2.0%포인트를 유지했다. 이번 에서 가장 주목한 지표는 연준 경제전망요약의 점도표다. 점도표는 연준 위원들이 각자의 금리 전망치를 점을 찍어 만든 표를 말한다. 이 중간값을 살펴보면 연준의 향후 정책 금리 향방을 가늠할 수 있다. 이에 따르면 연말 ',
   'link': 'https://www.donga.com/news/Economy/arti

In [20]:
hybrid_search(passage2) # 필터없이 2024년 전체 기사를 기반으로

[Document(metadata={'id': '125425689_0', 'title': '미연준, 기준금리 올해 1차례 인하 시사', 'date': 20240614, 'NER': '미국 연방준비제도 Fed 유럽 캐나다 미국 연방공개시장위원회 미 제롬 파월 의장'}, page_content='금리 7연속 동결… 5.5% 유지 파월 \'9월 인하\'도 배제 안해 미국 중앙은행 연방준비제도(Fed·연준)가 12일(현지 시간) 기준금리를 동결하고 연말까지 한 차례의 금리 인하를 시사했다. 유럽과 캐나다가 금리 인하로 \'피벗\'(정책 전환)에 나섰지만, 미국은 9월 금리 인하 가능성을 낮추며 고금리 유지에 무게를 뒀다. 연준은 이날 연방공개시장위원회(FOMC) 정례회의를 열고 "7차례 연속 금리를 동결해 미 기준금리를 5.25~5.50%로 유지한다"고 밝혔다. 연말 금리 중간값은 5.1%(5.0~5.25%)로 현 금리보다 0.25%포인트 낮은 수준이다. 이날 오전 발표된 5월 소비자물가지수(CPI) 상승률이 3.3%로 시장 전망치(3.4%)를 하회하는 등 물가상승률 둔화 시그널이 나왔지만, 연준은 기존 3차례 인하에서 1차례 인하로 인하 전망 폭을 오히려 축소한 것이다. 제롬 파월 연준 의장은 '),
 Document(metadata={'id': '123651783_0', 'title': '한은 "상반기 금리인하 어려워"… 9연속 3.5% 동결', 'date': 20240222, 'NER': '한국은행 미국 연방준비제도 Fed 위원 한미 이창용 한국은행 총재 한은 금통위원'}, page_content='국내외 소비자 물가 고공행진 감안 부동산 가격-가계부채 자극 경계 성장률 전망치는 2.1%로 유지 미연준도 기준금리 인하 신중론 고금리·고물가의 여파로 국내외 소비자 물가가 고공 행진하면서 올 상반기(1~6월) 내 기준금리 인하를 기대하기 어렵게 됐다. 지난해 2월 이후 9번 연속 금리를 동결한 한국은행은 상반기 금리 인하에 선을 그었다. 미국 연방준비제도(Fed·연